Command line call:

```
$ ctapipe-stage1 --config stage1.json --progress --overwrite
```

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

%matplotlib widget

In [ ]:
plt.rcParams['figure.constrained_layout.use'] = True
plt.rcParams['figure.dpi'] = 150

In [ ]:
from ctapipe.io import read_table


images = read_table('./LST-1.Run02965.0000_first400.dl1.h5', '/dl1/event/telescope/images/tel_001')
images[:5]

In [ ]:
trigger = read_table('./LST-1.Run02965.0000_first400.dl1.h5', '/dl1/event/subarray/trigger')
trigger[:5]

In [ ]:
from astropy.table import join

# combine the trigger info and images into one tables
images = join(images, trigger, keys=('obs_id', 'event_id'))
images[:5]

In [ ]:
from ctapipe.containers import EventType

is_flatfield = images['event_type'] == EventType.FLATFIELD.value
images[is_flatfield]

In [ ]:
plt.figure()
plt.hist(images[is_flatfield]['image'].ravel(), bins=100)

None

In [ ]:
mean_flatfield_image = images[is_flatfield]['image'].mean(axis=0)
mean_flatfield_image

In [ ]:
too_low, = np.nonzero(mean_flatfield_image < 30)
too_low

In [ ]:
from ctapipe.visualization import CameraDisplay
from ctapipe.instrument import SubarrayDescription

subarray = SubarrayDescription.from_hdf('./LST-1.Run02965.0000_first400.dl1.h5')

camera = subarray.tel[1].camera.geometry

fig, ax = plt.subplots(constrained_layout=True)
disp = CameraDisplay(camera, image=mean_flatfield_image, ax=ax, cmap='inferno')
disp.add_colorbar()
disp.highlight_pixels(too_low, color='g', linewidth=2)